In [1]:
%%configure -f

{"conf":
     {"spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.pyspark.virtualenv.bin.path":"/home/hadoop/epidemics-venv/bin",
      "spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.dynamicAllocation.enabled":"true",
      "spark.executor.memory":"16g",
      "spark.driver.memory":"16g",
      "spark.executor.cores":"6",
      "spark.driver.cores":"6",
      "spark.driver.maxResultSize":"20g",
      "livy.server.session.timeout-check" :"false"}
}

In [2]:
import pandas as pd
import geopandas as gpd
import json
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, shape, box
from shapely.ops import unary_union, transform

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, ArrayType

import daphme as dm
from daphme import cleaning

from sedona.spark import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1703459807261_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Example1").\
    getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Intro to DAPHME

In this script, we will demonstrate some of DAPHME's functionalities for analyzing human mobility data. Our analysis will focus on the month of February for the city of Philadelphia. Using a dataset that originally contains [60000] users. 

In our analysis, we will perform three different kinds of operations, which are common in the analyses in these types of data. 
* **Importing**, which optionally converts the datetime to a specified timezone and project the coordinates into ESPG:3857 (which has units in meters and might be appropriate for a local analysis not too far from the equator). 
* **Coarse Filtering**, where we subset to users to make computations for tractable. In particular, we filter out users with very few days/hours with activity, and keep TODO [users with "sufficient" pings in the area]. We also filter to users with activity in a given time period.
* **Persisting**

## Load Pings

Daphme can work with a single file in a number of formats (csv, parquet, txt) but its true utility comes when working with partitions of the data, which are often organized in a directory. In this example, we specify the path of the files to be loaded.

In [21]:
gravy_df = spark.read.parquet("s3://phl-pings/gravy/")

gravy_df = gravy_df. \
    withColumn("latitude", F.col("geohashlatitude").cast(DoubleType())). \
    withColumn("longitude", F.col("geohashlongitude").cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#cuebiq_df = spark.read.options(header='True', inferSchema='True', delimiter=',').parquet("s3://phl-pings/cuebiq-jan-mar/")
#cuebiq_df.show()

In [17]:
def to_local_time(df: DataFrame, 
                  timezone_to: str,
                  timestamp_col: str = "timestamp") -> DataFrame:
    """
    Parameters
    ----------
    df: Spark dataframe with a column named timestamp_col, which contains timestamps
            
    timezone: a valid timezone identifier for the local timezone of the data. E.g.,"America/New_York", "UTC" 
    
    timestamp_col: (optional) name of the column containing timestamps
    
    Returns
    ----------
    Spark dataframe with additional columns
        'local_timestamp'
        'date'
        'date_hour'
        'day_of_week' (1 for Sunday, 2 for Monday, ..., 7 for Saturday)
    """
    
    # Convert timestamp to local timestamp
    df = df.withColumn(
        "local_timestamp",
        F.from_utc_timestamp(
            F.to_timestamp(F.col(timestamp_col) / 1000),  # divide by 1000 for milliseconds
            timezone_to  # convert from UTC to timezone specified by timezone_to
        ))

    # Add date, date_hour, and day_of_week columns
    df = df.withColumn(
        "date",
        F.to_date(F.col("local_timestamp"))
    ).withColumn(
        "date_hour",
        F.date_format(F.col("local_timestamp"), "yyyy-MM-dd HH")
    ).withColumn(
        "day_of_week",
        F.dayofweek(F.col("local_timestamp"))
    )

    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
gravy_df = to_local_time(gravy_df, 'America/New_York')
gravy_df = cleaning.clean_coords(gravy_df, spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#gravy_df.select("grid").distinct().count()
#gravy_df.printSchema()

In [11]:
gravy_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+----------------+-----------+-------------+----------------+--------------+------------+----------+-----------+---------+----------+-------------------+----------+-------------+-----------+--------------------+
|                grid|geohashlatitude|geohashlongitude|geohashnine|    timestamp|        timezone|     ipaddress|forensicflag|devicetype|recordcount| latitude| longitude|    local_timestamp|      date|    date_hour|day_of_week|      mercator_coord|
+--------------------+---------------+----------------+-----------+-------------+----------------+--------------+------------+----------+-----------+---------+----------+-------------------+----------+-------------+-----------+--------------------+
|00000006-23aa-3e4...|      39.947813|      -74.911673|  dr4er26bz|1583099437000|America/New_York| 73.76.110.202|       33536|      AAID|          1|39.947813|-74.911673|2471-08-30 22:26:10|2471-08-30|2471-08-30 22|          1|POINT (4858361.50...|
|000

## Coarse Filtering

Takes in a geometry and filters to pings within the given geometry

Load in a bounding box for Philadelphia

In [12]:
import boto3
from io import BytesIO

s3 = boto3.client('s3')
bucket_name = 'upenn-seas-wattscovid19lab'
object_key = 'paco/geometry/Census_Tracts_2010.geojson'

obj = s3.get_object(Bucket=bucket_name, Key=object_key)
geojson_data = json.load(BytesIO(obj['Body'].read()))

features = geojson_data['features']

polygons = [shape(feature["geometry"]).buffer(0) for feature in features if feature["properties"]["GEOID10"][:5] == "42101"]
phila_poly = unary_union(polygons).buffer(0.0015).simplify(0.0015)
phila_box = box(*phila_poly.bounds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
gravy_df = gravy_df.withColumn(
        "coordinate",
        F.expr("ST_MakePoint(longitude, latitude)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
testDF = gravy_df.limit(100)
testDF = testDF.select("grid", "local_timestamp", "coordinate", "mercator_coord")
testDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------------------+--------------------+
|                grid|    local_timestamp|          coordinate|      mercator_coord|
+--------------------+-------------------+--------------------+--------------------+
|00000006-23aa-3e4...|2020-03-01 16:50:37|POINT (-74.911673...|POINT (4858361.50...|
|00000013-ec62-3ef...|2019-10-16 05:36:06|POINT (-75.144017...|POINT (4867887.83...|
|00000013-ec62-3ef...|2019-10-16 07:36:39|POINT (-75.144017...|POINT (4867887.83...|
|00000013-ec62-3ef...|2019-10-16 10:36:16|POINT (-75.174015...|POINT (4870576.65...|
|00000013-ec62-3ef...|2019-10-27 22:34:02|POINT (-75.094707...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-27 23:36:07|POINT (-75.094707...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-28 02:36:09|POINT (-75.094707...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-28 03:36:09|POINT (-75.094707...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-28 05:36:06|POINT (-75.094707...|PO

In [39]:
# Obtain unique users with at least 1 ping in sample window
phila_box_wkt = phila_box.wkt

testDF.createOrReplaceTempView("gravy_df")

query = f"""
WITH UniqueIDs AS (
    SELECT DISTINCT grid
    FROM gravy_df
    WHERE ST_Contains(ST_GeomFromWKT('{phila_box_wkt}'), coordinate)
)

SELECT g.*
FROM gravy_df g
INNER JOIN UniqueIDs u ON g.grid = u.grid
"""

gravy_df_filtered = spark.sql(query)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
phila_box_wkt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'POLYGON ((-74.95426957576791 39.865972786981764, -74.95426957576791 40.13930486772445, -75.28146629180951 40.13930486772445, -75.28146629180951 39.865972786981764, -74.95426957576791 39.865972786981764))'

In [40]:
gravy_df_filtered.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------------+----------+--------------+
|grid|local_timestamp|coordinate|mercator_coord|
+----+---------------+----------+--------------+
+----+---------------+----------+--------------+